In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
import shap

import sys, os
sys.dont_write_bytecode = True
import conditions, analysis

import random
random.seed(1107)
np.random.seed(1107)

import warnings
warnings.filterwarnings('ignore')

PATH = 'Output_Figures'
os.makedirs(PATH, exist_ok=True)

PATH2 = 'Output_CSV'
os.makedirs(PATH2, exist_ok=True)

print('SHAP version:', shap.__version__)
print('SHAP PATH:')
print(shap.__path__)

In [7]:
condition = conditions.calc_condition()
Reaction, data_sheet_name = condition['Reaction'], condition['data_sheet_name']
pgm_model, add_model, supp_model = condition['pgm_model'], condition['add_model'], condition['supp_model']
pgm_num, add_num, supp_num = condition['pgm_num'], condition['add_num'], condition['supp_num']
target_name = condition['target_name']

converted = analysis.analysis_data_convert(condition, data_sheet_name, use_models=[pgm_model, add_model, supp_model], idx=None)
data, feat, target = converted['data'], converted['feat'], converted['target']

add_desc = converted['add_desc']
add_aw = converted['add_aw']
feat_cols = feat.columns

data_cols = conditions.data_columns(Reaction, condition)
elem_cols = data_cols['elem']
wt_cols = data_cols['wt']
data[wt_cols] = data[wt_cols].astype('float')

data = data.drop(['No.', 'CO Yield_%', 'CO formation rate_mmol min-1 gcat-1', 'Iteration', 'Catal prep', 'Reaction', 'Note'], axis=1)

data_sort = pd.DataFrame(index=np.arange(0, len(data), 1), columns=np.arange(0, len(data.columns), 1))
for i in range(len(data)):
    data_sort_elem = data.loc[i][[data.loc[i][wt_cols].sort_values(ascending=False).index[Adi].replace('_wt%', '') for Adi in range(add_num)]].reset_index().T.drop('index', axis=0)
    data_sort_wt = data.loc[i][wt_cols].sort_values(ascending=False).reset_index().T.drop('index', axis=0)
    data_sort.iloc[i] = pd.concat([data_sort_elem, data_sort_wt], axis=1)
data_sort = data_sort.rename(columns={
    0: f'{elem_cols[0]}', 1: f'{elem_cols[1]}', 2: f'{elem_cols[2]}', 3: f'{elem_cols[3]}', 4: f'{elem_cols[4]}',
    5: f'{wt_cols[0]}', 6: f'{wt_cols[1]}', 7: f'{wt_cols[2]}', 8: f'{wt_cols[3]}', 9: f'{wt_cols[4]}'
    }).reindex(columns=data.columns)
data_sort['Catalyst'] = 'Pt(3)/'

for i in range(len(data)):
    for j in range(add_num):
        if data_sort[elem_cols[j]][i] != 'H' and j == 0:
            data_sort['Catalyst'][i] += data_sort[elem_cols[j]][i]
        elif data_sort[elem_cols[j]][i] != 'H' and j != 0:
            data_sort['Catalyst'][i] += '-' + data_sort[elem_cols[j]][i]
        else:
            pass
        if data_sort[wt_cols[j]][i] >= 1:
            data_sort[wt_cols[j]][i] = data_sort[wt_cols[j]][i].astype(int)
            data_sort['Catalyst'][i] += '(' + data_sort[wt_cols[j]][i].astype(str) + ')'
        elif data_sort[wt_cols[j]][i] < 1 and data_sort[wt_cols[j]][i] > 0:
            data_sort['Catalyst'][i] += '(' + data_sort[wt_cols[j]][i].astype(str) + ')'
        else:
            pass
data_sort['Catalyst'] += '/TiO2'

Table_S4 = converted['data']
Table_S4 = Table_S4.drop(elem_cols, axis=1).drop(wt_cols, axis=1).drop(['Catal prep', 'Reaction', 'Note'], axis=1).rename(columns = {'No.':'ID'})
Table_S4.loc[:, ['CO Yield_%', 'CO formation rate_mmol min-1 gcat-1']] = Table_S4.loc[:, ['CO Yield_%', 'CO formation rate_mmol min-1 gcat-1']].round(2)
Table_S4.insert(loc = 1, column= 'Catalyst', value= data_sort['Catalyst'])
Table_S4.insert(loc = 3, column= 'CO selectivity_%', value= 100)

20220318 rwgs_250 all data


In [ ]:
model  = ExtraTreesRegressor(n_estimators=100, random_state=1107, n_jobs=-1)

os.makedirs(f'{PATH}/SHAP/waterfall/prop{add_model}', exist_ok=True)
data = converted['data']
data['CO formation rate_mmol min-1 gcat-1']=Table_S4['CO formation rate_mmol min-1 gcat-1']
target_name, ML_model = condition['target_name'], condition['ML_model']
model = ExtraTreesRegressor(n_estimators=100, random_state=1107, n_jobs=4)
model.fit(feat, target)

explainer = shap.Explainer(model, feat)
shap_values = explainer(feat)
shap_values.values = np.round(shap_values.values, 2)
shap_values.base_values = np.round(shap_values.base_values, 2)
shap_values.data = np.round(shap_values.data, 2)

plt.figure(facecolor='white', figsize =(8,6))

for i in range(len(data)):
    plt.figure(facecolor='white', figsize =(8,6))
    Cat_No = data['No.'][i]
    print('No.', data['No.'][i], Table_S4['Catalyst'][i], 'CO form rate:', round(data[target_name][i], 2))
    save_file_name = f'{PATH}/SHAP/waterfall/prop{add_model}/Cat_No_{Cat_No}_shap_waterfall_prop{add_model}.png'
    shap.plots.waterfall(shap_values[i], max_display=9, show=True, filename=save_file_name)